In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# Path and Support Threshold Setting

In [ ]:
path = "/content/drive/MyDrive/CSOE18_BDA_Assigment_1_107121045/"
min_support = 0.01
relative_min_support = 0.01
# To set the actual threshold, refer to 1st block of code under the "Reading Data and generating Candidates & Frequent Itemsets" section

# Functions Definitions




> File Handling Codes



In [ ]:
# Function to read data from a file and return a list of itemsets
def read_data_from_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Split the line into categories using ';' as the delimiter
            categories = line.strip().split(';')
            data.append(categories)
    return data

# Function to write a list of tuples to a file
def write_tuples_to_file(file_path, data_list):
    # Calculate the total count
    total_count = len(data_list)

    with open(file_path, "w") as file:
        # Write the header with the correct total count
        file.write(f"#Total count {total_count}\n")

        # Write each tuple's data
        for data in data_list:
            key, value = data
            # Check if the key is a frozenset and convert it to a comma-separated string
            if isinstance(key, frozenset):
                key_str = ', '.join(sorted(key))
            else:
                key_str = key
            file.write(f"{key_str}: {value}\n")

# Function to print data from a file
def print_from_file(file_path):
    with open(file_path, "r") as file:
        for line in file:
            # Print each line from the file
            print(line)




> Merge Sort



In [ ]:
# Function to merge-sort a dictionary
def merge_sort_dict(d):
    # Base case: If the dictionary has 1 or fewer items, return it as is
    if len(d) <= 1:
        return d

    # Find the middle index
    middle = len(d) // 2

    # Split the dictionary into two halves
    left_half = dict(list(d.items())[:middle])
    right_half = dict(list(d.items())[middle:])

    # Recursively merge-sort the left and right halves
    left_half = merge_sort_dict(left_half)
    right_half = merge_sort_dict(right_half)

    # Initialize an empty dictionary to store the merged result
    merged = {}

    # Initialize indices for the left and right halves
    left_idx, right_idx = 0, 0

    # Merge elements while comparing their keys
    while left_idx < len(left_half) and right_idx < len(right_half):
        left_key, right_key = list(left_half.keys())[left_idx], list(right_half.keys())[right_idx]

        # Compare the values associated with the keys
        if left_half[left_key] >= right_half[right_key]:
            merged[left_key] = left_half[left_key]
            left_idx += 1
        else:
            merged[right_key] = right_half[right_key]
            right_idx += 1

    # Add any remaining elements from the left and right halves
    while left_idx < len(left_half):
        left_key = list(left_half.keys())[left_idx]
        merged[left_key] = left_half[left_key]
        left_idx += 1

    while right_idx < len(right_half):
        right_key = list(right_half.keys())[right_idx]
        merged[right_key] = right_half[right_key]
        right_idx += 1

    # Return the merged dictionary
    return merged




> Itemset Generation



In [ ]:
def count_and_filter_itemsets_1(data, min_threshold):
    # Initialize a dictionary to store the candidate itemsets and their counts
    candidates = {}

    # Iterate through the transactional data
    for itemsets in data:
        for items in itemsets:
            # Generate a frozenset containing a single element (the item)
            item_key = frozenset({items})
            # Increment the count for the candidate itemset
            candidates[item_key] = candidates.get(item_key, 0) + 1

    # Create a dictionary containing only frequent itemsets (above min_threshold)
    freq_itemsets = {key: value for key, value in candidates.items() if value >= min_threshold}

    # Return both the candidates and frequent itemsets as a tuple
    return (candidates, freq_itemsets)


In [ ]:
def generate_itemsets_of_length_n(freq_itemsets, n):
    # Initialize an empty dictionary to store itemsets of length 'n'
    itemsets_n = {}

    if n == 2:
        # If 'n' is 2, generate unique combinations of length 2 from freq_itemsets
        for itemset1 in freq_itemsets.keys():
            for itemset2 in freq_itemsets.keys():
                # Ensure itemset1 and itemset2 are distinct
                if itemset1 != itemset2:
                    # Combine itemset1 and itemset2 and create a frozenset
                    pair_key = frozenset(itemset1.union(itemset2))
                    # Check if the length of the combined itemset is 'n'
                    if len(pair_key) == n:
                        # Add the pair_key to the itemsets_n dictionary with a value of 0
                        itemsets_n[pair_key] = 0

    if n == 3:
        # If 'n' is 3, generate unique combinations of length 3 from freq_itemsets
        for key1 in freq_itemsets.keys():
            for key2 in freq_itemsets.keys():
                # Ensure key1 and key2 are distinct
                if key1 != key2:
                    # Combine key1 and key2 to create a new itemset
                    itemset_3 = key1.union(key2)
                    # Check if the length of the combined itemset is 'n'
                    if len(itemset_3) == 3:
                        # Add the itemset_3 to the itemsets_n dictionary with a value of 0
                        itemsets_n[itemset_3] = 0

    if n == 4:
        # If 'n' is 4, generate unique combinations of length 4 from freq_itemsets
        for key1 in freq_itemsets.keys():
            for key2 in freq_itemsets.keys():
                for key3 in freq_itemsets.keys():
                    # Ensure key1, key2, and key3 are distinct
                    if key1 != key2 or key1 != key3 or key2 != key3:
                        # Create a temporary itemset by combining key1 and key2
                        tmp = key1.union(key2)
                        # Create a new itemset by combining tmp and key3
                        itemset_4 = tmp.union(key3)
                        # Check if the length of the combined itemset is 'n'
                        if len(itemset_4) == 4:
                            # Add the itemset_4 to the itemsets_n dictionary with a value of 0
                            itemsets_n[itemset_4] = 0

    return itemsets_n




> Candidate Itemsets Generation



In [ ]:
def counting_from_data(data, itemsets_dict):
    # Initialize an empty dictionary to store candidate sets
    candidate_sets = {}

    # Iterate through each itemset in the data
    for itemset in data:
        # Convert the itemset into a set for efficient membership checking
        itemset_items = set(itemset)

        # Iterate through the itemsets in itemsets_dict
        for candidates in itemsets_dict.keys():
            # Check if the candidates are a subset of the current itemset
            if candidates.issubset(itemset_items):
                # Increment the count of the candidate set in candidate_sets
                candidate_sets[candidates] = candidate_sets.get(candidates, 0) + 1

    return candidate_sets




> Closed Itemset Generation



In [ ]:
def mine_closed_frequent_itemsets(all_, relative_min_support):
    # Step 1: Filter frequent itemsets by relative minimum support
    filtered_itemsets = {itemset: support for itemset, support in all_.items() if support >= relative_min_support}

    closed_frequent_itemsets = {}

    # Step 2: Check each frequent itemset for closure
    for itemset1, support1 in filtered_itemsets.items():
        is_closed = True
        for itemset2, support2 in filtered_itemsets.items():
            # Check if itemset1 is a proper subset of itemset2 and has the same support
            if itemset1 != itemset2 and itemset1.issubset(itemset2) and support1 == support2:
                is_closed = False
                break
        if is_closed:
            closed_frequent_itemsets[itemset1] = support1

    # Step 3: Sort closed frequent itemsets by support count in descending order
    sorted_closed_frequent_itemsets = merge_sort_dict(closed_frequent_itemsets)

    return sorted_closed_frequent_itemsets


# Reading Data and Generating Candidates & Frequent Itemsets



> Itemsets of Length 1



In [ ]:
# Read data from a file and store it in the 'data' variable
data = read_data_from_file(path + "categories.txt")

# Calculate the minimum support threshold based on the specified 'min_support' parameter
min_threshold = min_support * len(data)     #Leads to considering min_threshold value =771.85 i.e. 772
#min_threshold = 771
# Count and filter itemsets of length 1 from the data
(candidate_set_1, freq_itemsets_1) = count_and_filter_itemsets_1(data, min_threshold)

# Merge-sort the frequent itemsets of length 1
sorted_data = merge_sort_dict(freq_itemsets_1)

# Print the sorted frequent itemsets and their length
print("Frequent Itemsets of Length 1:\n",sorted_data, "\n\nLength of frequent itemset of length 1:\t", len(sorted_data))


Frequent Itemsets of Length 1:
 {frozenset({'Restaurants'}): 25071, frozenset({'Shopping'}): 11233, frozenset({'Food'}): 9250, frozenset({'Beauty & Spas'}): 6583, frozenset({'Health & Medical'}): 5121, frozenset({'Nightlife'}): 5088, frozenset({'Home Services'}): 4785, frozenset({'Bars'}): 4328, frozenset({'Automotive'}): 4208, frozenset({'Local Services'}): 3468, frozenset({'Active Life'}): 3103, frozenset({'Fashion'}): 3078, frozenset({'Event Planning & Services'}): 2975, frozenset({'Fast Food'}): 2851, frozenset({'Pizza'}): 2657, frozenset({'Mexican'}): 2515, frozenset({'Hotels & Travel'}): 2495, frozenset({'American (Traditional)'}): 2416, frozenset({'Sandwiches'}): 2364, frozenset({'Arts & Entertainment'}): 2271, frozenset({'Coffee & Tea'}): 2199, frozenset({'Hair Salons'}): 2091, frozenset({'Italian'}): 1848, frozenset({'Burgers'}): 1774, frozenset({'Auto Repair'}): 1716, frozenset({'Doctors'}): 1694, frozenset({'Nail Salons'}): 1667, frozenset({'Chinese'}): 1629, frozenset({'Ame



> Itemsets of Length 2



In [ ]:
# Generate itemsets of length 2 from the frequent itemsets of length 1
itemsets_2 = generate_itemsets_of_length_n(freq_itemsets_1, 2)

# Print the length of itemsets of length 2
print("Length of itemsets of length 2:\t", len(itemsets_2), "\n\n")

# Generate candidate itemsets of length 2 by checking support in the data
candidate_set_2 = counting_from_data(data, itemsets_2)

# Print the length of candidate itemsets of length 2
print("Length of candidate itemsets of length 2:\t", len(candidate_set_2), "\n\n")

# Filter candidate itemsets of length 2 using minimum threshold support
freq_itemsets_2 = {key: value for key, value in candidate_set_2.items() if value >= min_threshold}

# Merge-sort the frequent itemsets of length 2
sorted_data_2 = merge_sort_dict(freq_itemsets_2)

# Print the sorted frequent itemsets of length 2 and their length
print("Frequent Itemsets of Length 2:\n",sorted_data_2, "\n\nLength of frequent itemset of length 2:\t", len(sorted_data_2))


Length of itemsets of length 2:	 1225 


Length of candidate itemsets of length 2:	 598 


Frequent Itemsets of Length 2:
 {frozenset({'Nightlife', 'Bars'}): 4328, frozenset({'Fashion', 'Shopping'}): 3078, frozenset({'Restaurants', 'Fast Food'}): 2851, frozenset({'Restaurants', 'Pizza'}): 2657, frozenset({'Restaurants', 'Nightlife'}): 2533, frozenset({'Restaurants', 'Mexican'}): 2515, frozenset({'Restaurants', 'Bars'}): 2423, frozenset({'Restaurants', 'American (Traditional)'}): 2416, frozenset({'Restaurants', 'Sandwiches'}): 2364, frozenset({'Food', 'Coffee & Tea'}): 2199, frozenset({'Food', 'Restaurants'}): 2101, frozenset({'Beauty & Spas', 'Hair Salons'}): 2091, frozenset({'Restaurants', 'Italian'}): 1848, frozenset({'Restaurants', 'Burgers'}): 1774, frozenset({'Automotive', 'Auto Repair'}): 1716, frozenset({'Health & Medical', 'Doctors'}): 1694, frozenset({'Nail Salons', 'Beauty & Spas'}): 1667, frozenset({'Restaurants', 'Chinese'}): 1629, frozenset({'American (New)', 'Restaurants'



> Itemsets of Length 3



In [ ]:
# Generate itemsets of length 3 from the frequent itemsets of length 2
itemsets_3 = generate_itemsets_of_length_n(freq_itemsets_2, 3)

# Print the length of itemsets of length 3
print("Length of itemsets of length 3:\t", len(itemsets_3), "\n\n")

# Generate candidate itemsets of length 3 by checking support in the data
candidate_set_3 = counting_from_data(data, itemsets_3)

# Print the length of candidate itemsets of length 3
print("Length of candidate itemsets of length 3:\t", len(candidate_set_3), "\n\n")

# Filter candidate itemsets of length 3 using minimum threshold support
freq_itemsets_3 = {key: value for key, value in candidate_set_3.items() if value >= min_threshold}

# Merge-sort the frequent itemsets of length 3
sorted_data_3 = merge_sort_dict(freq_itemsets_3)

# Print the sorted frequent itemsets of length 3 and their length
print("Frequent Itemsets of Length 3:\n",sorted_data_3, "\n\nLength of frequent itemset of length 3:\t", len(sorted_data_3))


Length of itemsets of length 3:	 151 


Length of candidate itemsets of length 3:	 141 


Frequent Itemsets of Length 3:
 {frozenset({'Restaurants', 'Nightlife', 'Bars'}): 2423, frozenset({'Hotels', 'Event Planning & Services', 'Hotels & Travel'}): 1431, frozenset({'Fashion', 'Shopping', "Women's Clothing"}): 1138, frozenset({'Nightlife', 'Pubs', 'Bars'}): 874, frozenset({'Dentists', 'Health & Medical', 'General Dentistry'}): 823, frozenset({'Sports Bars', 'Nightlife', 'Bars'}): 818, frozenset({'Restaurants', 'Burgers', 'Fast Food'}): 774} 

Length of frequent itemset of length 3:	 7




> Itemsets of length 4



In [ ]:
# Generate itemsets of length 4 from the frequent itemsets of length 3
itemsets_4 = generate_itemsets_of_length_n(freq_itemsets_3, 4)

# Print the length of itemsets of length 4
print("Length of itemsets of length 4:\t", len(itemsets_4), "\n\n")

# Generate candidate itemsets of length 4 by checking support in the data
candidate_set_4 = counting_from_data(data, itemsets_4)

# Print the candidate set of length 4
print("The candidate set of length 4:\n", candidate_set_4, "\n\n")

# Print the length of candidate itemsets of length 4
print("Length of candidate itemsets of length 4:\t", len(candidate_set_4), "\n\n")

# Filter candidate itemsets of length 4 using minimum threshold support
freq_itemsets_4 = {key: value for key, value in candidate_set_4.items() if value >= min_threshold}

# Merge-sort the frequent itemsets of length 4
sorted_data_4 = merge_sort_dict(freq_itemsets_4)

# Print the sorted frequent itemsets of length 4 and their length
print("Frequent Itemsets of Length 4:\n",sorted_data_4, "\n\nLength of frequent itemset of length 4:\t", len(sorted_data_4))


Length of itemsets of length 4:	 3 


The candidate set of length 4:
 {frozenset({'Restaurants', 'Sports Bars', 'Nightlife', 'Bars'}): 596, frozenset({'Restaurants', 'Nightlife', 'Pubs', 'Bars'}): 431, frozenset({'Sports Bars', 'Nightlife', 'Pubs', 'Bars'}): 90} 


Length of candidate itemsets of length 4:	 3 


Frequent Itemsets of Length 4:
 {} 

Length of frequent itemset of length 4:	 0


As there is no frequent itemset of length 4, we cannot generate candidate itemsets of length 5 which is supported by the **Anti-monotone Property of Support.**

# Closed Frequent Itemsets


In [ ]:
# Create a list of patterns for different lengths of frequent itemsets
patterns_all_data = [sorted_data, sorted_data_2, sorted_data_3, sorted_data_4]

# Combine all frequent itemsets from different lengths into a single dictionary
all_frequent_itemsets = {key: value for data in patterns_all_data for key, value in data.items()}

# Calculate and print the total number of frequent itemsets in the combined dictionary
print("Total number of frequent itemsets:", len(all_frequent_itemsets))

# Mine closed frequent itemsets from the combined dictionary with relative minimum support
closed_frequent_itemsets = mine_closed_frequent_itemsets(all_frequent_itemsets, relative_min_support)

# Calculate and print the total number of closed frequent itemsets
print("\n\nAll Closed Frequent Itemsets :\n",closed_frequent_itemsets,"\n\nTotal number of closed frequent itemsets:", len(closed_frequent_itemsets))


Total number of frequent itemsets: 101


All Closed Frequent Itemsets :
 {frozenset({'Restaurants'}): 25071, frozenset({'Shopping'}): 11233, frozenset({'Food'}): 9250, frozenset({'Beauty & Spas'}): 6583, frozenset({'Health & Medical'}): 5121, frozenset({'Nightlife'}): 5088, frozenset({'Home Services'}): 4785, frozenset({'Nightlife', 'Bars'}): 4328, frozenset({'Automotive'}): 4208, frozenset({'Local Services'}): 3468, frozenset({'Active Life'}): 3103, frozenset({'Fashion', 'Shopping'}): 3078, frozenset({'Event Planning & Services'}): 2975, frozenset({'Restaurants', 'Fast Food'}): 2851, frozenset({'Restaurants', 'Pizza'}): 2657, frozenset({'Restaurants', 'Nightlife'}): 2533, frozenset({'Restaurants', 'Mexican'}): 2515, frozenset({'Hotels & Travel'}): 2495, frozenset({'Restaurants', 'Nightlife', 'Bars'}): 2423, frozenset({'Restaurants', 'American (Traditional)'}): 2416, frozenset({'Restaurants', 'Sandwiches'}): 2364, frozenset({'Arts & Entertainment'}): 2271, frozenset({'Food', 'Coffee & 

# Writing into Files

In [ ]:
#Write sorted frequent itemsets of length 1 into a file
write_tuples_to_file(path + "patterns_1.txt", sorted_data.items())

# Write all sorted frequent itemsets for the given data into a file
write_tuples_to_file(path + "patterns_all.txt", [(key, value) for data in patterns_all_data for key, value in data.items()])

#Write all sorted closed frequent itemsets for given all frequent itemsets into a file
write_tuples_to_file(path + "patterns_close.txt", closed_frequent_itemsets.items())



> Cross-verifictation



In [ ]:
# Print the contents of "patterns_1.txt" file
print_from_file(path + "patterns_1.txt")
print("\n\n\n\n")  # Add extra newlines for separation

# Print the contents of "patterns_all.txt" file
print_from_file(path + "patterns_all.txt")
print("\n\n\n\n")  # Add extra newlines for separation

# Print the contents of "patterns_close.txt" file
print_from_file(path + "patterns_close.txt")


#Total count 50

Restaurants: 25071

Shopping: 11233

Food: 9250

Beauty & Spas: 6583

Health & Medical: 5121

Nightlife: 5088

Home Services: 4785

Bars: 4328

Automotive: 4208

Local Services: 3468

Active Life: 3103

Fashion: 3078

Event Planning & Services: 2975

Fast Food: 2851

Pizza: 2657

Mexican: 2515

Hotels & Travel: 2495

American (Traditional): 2416

Sandwiches: 2364

Arts & Entertainment: 2271

Coffee & Tea: 2199

Hair Salons: 2091

Italian: 1848

Burgers: 1774

Auto Repair: 1716

Doctors: 1694

Nail Salons: 1667

Chinese: 1629

American (New): 1593

Home & Garden: 1586

Pets: 1497

Fitness & Instruction: 1442

Hotels: 1431

Real Estate: 1424

Grocery: 1424

Breakfast & Brunch: 1369

Dentists: 1195

Specialty Food: 1150

Women's Clothing: 1138

Bakeries: 1115

Professional Services: 1025

Ice Cream & Frozen Yogurt: 1018

Cafes: 1002

Financial Services: 875

Pubs: 874

Pet Services: 870

Japanese: 848

General Dentistry: 823

Sports Bars: 818

Sushi Bars: 798






#Total

# Cross Checking with Apriori Library in mlextend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import warnings

# Suppress DeprecationWarnings
warnings.simplefilter("ignore", DeprecationWarning)

# Create a DataFrame with one-hot encoding
oht = pd.DataFrame(pd.Series(data).str.join('|').str.get_dummies('|'))

In [ ]:
# Suppress DeprecationWarnings
warnings.simplefilter("ignore", DeprecationWarning)

# Frequent itemsets of length 1
all_frequent_itemsets_lib = apriori(oht, min_support=min_support, use_colnames=True)
# Filter frequent itemsets of length 1
frequent_itemsets_1_lib = all_frequent_itemsets_lib[all_frequent_itemsets_lib['itemsets'].apply(lambda x: len(x) == 1)]

print("Frequent Itemsets of Length 1:")
print(frequent_itemsets_1_lib,"\n\n\n" ,len(frequent_itemsets_1_lib))

Frequent Itemsets of Length 1:
     support                     itemsets
0   0.040202                (Active Life)
1   0.020639             (American (New))
2   0.031301     (American (Traditional))
3   0.029423       (Arts & Entertainment)
4   0.022232                (Auto Repair)
5   0.054518                 (Automotive)
6   0.014446                   (Bakeries)
7   0.056073                       (Bars)
8   0.085289              (Beauty & Spas)
9   0.017737         (Breakfast & Brunch)
10  0.022984                    (Burgers)
11  0.012982                      (Cafes)
12  0.021105                    (Chinese)
13  0.028490               (Coffee & Tea)
14  0.015482                   (Dentists)
15  0.021947                    (Doctors)
16  0.038544  (Event Planning & Services)
17  0.039878                    (Fashion)
18  0.036937                  (Fast Food)
19  0.011336         (Financial Services)
20  0.018682      (Fitness & Instruction)
21  0.119842                       (Food)
22 

In [ ]:
# Suppress DeprecationWarnings
warnings.simplefilter("ignore", DeprecationWarning)

print("\nAll Frequent Itemsets:")
print(all_frequent_itemsets_lib,"\n\n\n" ,len(all_frequent_itemsets_lib))


All Frequent Itemsets:
      support                                           itemsets
0    0.040202                                      (Active Life)
1    0.020639                                   (American (New))
2    0.031301                           (American (Traditional))
3    0.029423                             (Arts & Entertainment)
4    0.022232                                      (Auto Repair)
..        ...                                                ...
96   0.010598                     (Nightlife, Sports Bars, Bars)
97   0.010028                  (Restaurants, Burgers, Fast Food)
98   0.010663    (Health & Medical, Dentists, General Dentistry)
99   0.018540  (Hotels, Event Planning & Services, Hotels & T...
100  0.014744              (Fashion, Shopping, Women's Clothing)

[101 rows x 2 columns] 


 101


In [ ]:
# Suppress DeprecationWarnings
warnings.simplefilter("ignore", DeprecationWarning)

# Closed frequent itemsets
closed_itemsets_lib = association_rules(all_frequent_itemsets_lib, metric="support", min_threshold=relative_min_support)
closed_itemsets_lib = closed_itemsets_lib[closed_itemsets_lib['confidence'] == 1]  # Filter by lift = 1 for closed itemsets
print("\nClosed Frequent Itemsets:")
print(closed_itemsets_lib,"\n\n\n" ,len(closed_itemsets_lib))


Closed Frequent Itemsets:
                               antecedents  \
1                  (Fitness & Instruction)   
2                         (American (New))   
5                 (American (Traditional))   
7                            (Auto Repair)   
9                               (Bakeries)   
11                                  (Bars)   
12                                  (Pubs)   
16                           (Sports Bars)   
19                           (Hair Salons)   
20                           (Nail Salons)   
23                    (Breakfast & Brunch)   
27                               (Burgers)   
29                                 (Cafes)   
31                               (Chinese)   
33                          (Coffee & Tea)   
35                     (General Dentistry)   
37                              (Dentists)   
39                               (Doctors)   
41                                (Hotels)   
44                               (Fashion)   
47     

# Final Comparision

In [ ]:
# Suppress DeprecationWarnings
warnings.simplefilter("ignore", DeprecationWarning)

print(f"Considering - \t\t\tmin_support :\t{min_support}\t\t min_threshold :\t{min_threshold} \n")

print("Freq_itemsets_1\t\t" ,len(sorted_data),"\tAll_itemsets\t\t",len(all_frequent_itemsets),"\tClosed Freq Itemsets\t\t", len(closed_frequent_itemsets))
print("Freq_itemsets_1_lib\t" ,len(frequent_itemsets_1_lib),"\tAll_itemsets_lib\t",len(all_frequent_itemsets_lib),"\tClosed Freq Itemsets_lib\t", len(closed_itemsets_lib))

Considering - 			min_support :	0.01		 min_threshold :	771.85 

Freq_itemsets_1		 50 	All_itemsets		 101 	Closed Freq Itemsets		 55
Freq_itemsets_1_lib	 50 	All_itemsets_lib	 101 	Closed Freq Itemsets_lib	 56


The slight change in closed frequent itemsets is due to the decimal error while calculating min_threshold.
Also if threshold is set at 771 instead of 771.85 [772] then also slight errors in results are seen.

A slight difference is observed in either All itemsets or Closed Freq Itemsets while comparing the two approaches is due to the following factors:-

*   When min_threshold is set at 771.85 i.e. 772, the closed freq itemsets count is 55 instead of 56 due to error in calculation of confidence (after 8 decimal places)
*   When min_threshold is set to 771, the all itemsets detects 2 more than required as there are exactly 2 itemsets which have threshold of 771 but fails in case od approximated support/confidence (8 digit decimal errors)

These errors can either be resolved by using datatypes of higher bits say long or double but at the cost of memory and computaion power/efficiency/speed.





> The execution of main code takes around 15 seconds, while as library takes around 30 seconds.



# References




1.   Various webpages including GitHub, Kaggle, StackOverFlow, among others related to coding of Apriori Algorithm. Some of which are:-

*   [Apriori](https://www.kaggle.com/code/nandinibagga/apriori-algorithm)
*   [Market Basket Analysis](https://www.kaggle.com/code/ozlemilgun/market-basket-analysis-with-apriori-algorithm)
*   [Groceries Association Rule](https://www.kaggle.com/code/cerolacia/groceries-asscociation-rule-apriori-algorithm)
*   From [Medium](https://medium.com/linkit-intecs/apriori-algorithm-in-data-mining-part-2-590d58e0998b)
*   From [GfG- Geeks for Geeks](https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/)
*   From [Intellipaat](https://intellipaat.com/blog/data-science-apriori-algorithm/)
*   From [Javapoint](https://www.javatpoint.com/implementing-apriori-algorithm-in-python)
*   From [towardsdatascience](https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6)
*   From [section.io](https://www.section.io/engineering-education/apriori-algorithm-in-python/)
*   From [GitHub](https://github.com/topics/apriori-algorithm-python)

2.   AI tools like ChatGPT, Google Bard, Bing Chat, etc only used for the sake of commenting ONLY. Entire code has been written by
**Mr. Kshitij Kanade, 107121045, EEE Dept., NIT Trichy.**

3.  Class notes as well as personal guidance by Prof. C. Oswald, CSE Dept., NIT Trichy.

4.  Reference Books Like Massive Data Sets, etc provided under CSOE18 Big Data Analytics Open Elective Course at NIT Trichy.